In [40]:
import pandas as pd
import numpy as np

In [41]:
#VParamètres
srcFolder = "../data/raw/data.gouv/"

In [42]:
df = pd.read_parquet(f"{srcFolder}meteo.parquet")

In [43]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40016341 entries, 0 to 40016340
Data columns (total 7 columns):
 #   Column      Dtype  
---  ------      -----  
 0   Unnamed: 0  int64  
 1   TX          float64
 2   TN          float64
 3   LAT         float64
 4   LON         float64
 5   AAAAMMJJ    int64  
 6   ALTI        int64  
dtypes: float64(4), int64(3)
memory usage: 2.1 GB


In [44]:
def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calcule la distance en kilomètres entre deux points géographiques
    en utilisant la formule de Haversine
    """
    # Rayon de la Terre en km
    R = 6371.0
    
    # Conversion en radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)
    
    # Différences
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    
    # Formule de Haversine
    a = np.sin(dlat/2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    distance = R * c
    
    return distance

In [45]:
#Soit une date au 1er février
dateEnd = 20100201
dateFin = 20100101

latA = 46.0
longA = 5.0

test = df[(df["AAAAMMJJ"] <= dateEnd) & (df["AAAAMMJJ"] >= dateFin)].sort_values("AAAAMMJJ").copy()


test['distance'] = haversine_distance(latA, longA, test['LAT'], test['LON'])


points_plus_proches = df.loc[test.groupby('AAAAMMJJ')['distance'].idxmin()]
points_plus_proches

points_plus_proches['T_moy'] = (points_plus_proches['TX'] + points_plus_proches['TN']) / 2

T_base = 5
points_plus_proches['GDD'] = points_plus_proches['T_moy'].apply(lambda x: max(0, x - T_base))


points_plus_proches['GDD_cumul'] = points_plus_proches['GDD'].cumsum()

points_plus_proches['jour_n'] = range(1, len(points_plus_proches) + 1)
points_plus_proches



,Unnamed: 0,TX,TN,LAT,LON,AAAAMMJJ,ALTI,T_moy,GDD,GDD_cumul,jour_n
216143,567246,5.5,4.5,46.038500,5.044333,20100101,280,5.00,0.00,0.00,1
18497437,1079237,3.5,1.0,45.380667,4.750167,20100102,140,2.25,0.00,0.00,2
12900872,308255,4.3,0.4,44.134833,4.708333,20100103,30,2.35,0.00,0.00,3
3297602,1470081,2.5,0.3,44.669500,4.360667,20100104,283,1.40,0.00,0.00,4
2723902,103074,4.5,0.4,44.712167,4.352167,20100105,600,2.45,0.00,0.00,5
11161621,610855,4.0,0.2,44.581167,4.733000,20100106,73,2.10,0.00,0.00,6
8058699,201370,3.3,0.5,45.146667,1.473333,20100107,115,1.90,0.00,0.00,7
3297604,1470085,2.0,0.7,44.669500,4.360667,20100108,283,1.35,0.00,0.00,8
2169370,569308,2.5,0.5,44.090667,6.852333,20100109,788,1.50,0.00,0.00,9
2169371,569309,2.5,0.5,44.090667,6.852333,20100110,788,1.50,0.00,0.00,10


In [49]:

from datetime import datetime

#Version créant N lignes en fonction d'une ligne de données
columns = ['latitude', 'longitude', 'n_avant_floraison', 'jour_n', 'temps_thermique']
dfFinal = pd.DataFrame(columns=columns)


#Soit une date au 1er mlai
dateEnd = 20100501
dateFin = 20100101

latA = 46.0
longA = 5.0

test = df[(df["AAAAMMJJ"] <= dateEnd) & (df["AAAAMMJJ"] >= dateFin)].sort_values("AAAAMMJJ").copy()


test['distance'] = haversine_distance(latA, longA, test['LAT'], test['LON'])


points_plus_proches = df.loc[test.groupby('AAAAMMJJ')['distance'].idxmin()]
points_plus_proches

points_plus_proches['T_moy'] = (points_plus_proches['TX'] + points_plus_proches['TN']) / 2

T_base = 5
points_plus_proches['GDD'] = points_plus_proches['T_moy'].apply(lambda x: max(0, x - T_base))


points_plus_proches['GDD_cumul'] = points_plus_proches['GDD'].cumsum()

points_plus_proches['jour_n'] = range(1, len(points_plus_proches) + 1)

col_latitude = []
col_longitude = []
col_n_avant_floraison = []
col_jour_n = []
col_temps_thermique = []

jour_floraison = len(points_plus_proches)+1

def jour_de_annee(date_str):
    """
    Calcule le numéro du jour dans l'année
    date_str: chaîne au format "AAAAMMJJ"
    retourne: numéro du jour (1-366)
    """
    date = datetime.strptime(date_str, "%Y%m%d")
    return date.timetuple().tm_yday

jour_n_theorique = jour_de_annee(str(dateEnd))
print(jour_n_theorique)

points_plus_proches = points_plus_proches.reset_index(drop=True)
for index, row in points_plus_proches.iterrows():
    col_latitude.append(latA)
    col_longitude.append(longA)
    col_n_avant_floraison.append(jour_floraison-index)
    col_jour_n.append(index+1)
    col_temps_thermique.append(row['GDD_cumul'])

dfAdd = pd.DataFrame({
    'latitude' : col_latitude,
    'longitude' : col_longitude,
    'n_avant_floraison' : col_n_avant_floraison,
    'jour_n' : col_jour_n,
    'temps_thermique' : col_temps_thermique
})
dfFinal = pd.concat([dfFinal, dfAdd])

dfFinal


121


/tmp/ipykernel_3823/559151851.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dfFinal = pd.concat([dfFinal, dfAdd])


,latitude,longitude,n_avant_floraison,jour_n,temps_thermique
0,46.0,5.0,122,1,0.00
1,46.0,5.0,121,2,0.00
2,46.0,5.0,120,3,0.00
3,46.0,5.0,119,4,0.00
4,46.0,5.0,118,5,0.00
...,...,...,...,...,...
116,46.0,5.0,6,117,332.85
117,46.0,5.0,5,118,344.50
118,46.0,5.0,4,119,359.20
119,46.0,5.0,3,120,370.05
